# Climate Outlook

```{admonition} **Valid for:**
:class: attention
October-November-December 2023
```
## Short Range Outlook

Let's start off looking at what to expect for the next 10 days. The plots below show forecasted weather parameters for the Grouse Mountain area for October 3-12, 2023. Note, the model elevation at this grid point is only 267 m. This forecast uses the Canadian Global Deterministic Prediction System (GDPS). 

Conditions should be dry and relatively warm until Oct 9. At this point, the ridge of high pressure breaks down and a strong weather system moves in spreading moderate to heavy rain. Conditions will most likely be cool and unsettled for a few days after this storm. 

In [186]:
#Import libraries
import os
import pandas as pd
import subprocess
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import json

In [187]:
os.chdir('C://Users/pmarshal/Documents/Climate-Outlook/monthly-climate')
os.getcwd()

'C:\\Users\\pmarshal\\Documents\\Climate-Outlook\\monthly-climate'

In [188]:
# Import dataset
meteo = pd.read_csv('data/SpotWxForecast.csv', parse_dates=['DATETIME', 'DATE']) 

In [189]:
meteo['PC1'] = meteo['APCP'].diff()
meteo['PC1'].fillna(0, inplace=True)
meteo = meteo.assign(Zero='0')

In [190]:
precip = alt.Chart(meteo).mark_bar(color='green', size=10).encode(
    alt.X('DATETIME:T', title=None, axis=alt.Axis(labelAngle=0, format='%e %b')),
    alt.Y('PC1:Q', title = '3-hr Precipitation (mm)'),
    tooltip=[alt.Tooltip('DATE', title="Date"), alt.Tooltip('TIME', title="Time"), alt.Tooltip('PC1', title="3-Hour Precip")]
).properties(width=700, height=130, title='3hr-precip (green bars) and accumulated precip (blue line)')

In [191]:
precip_accum = alt.Chart(meteo).mark_line(color='blue', opacity=0.6).encode(
    alt.X('DATETIME:T', title=None, axis=alt.Axis(labelAngle=0, format='%e %b')),
    alt.Y('APCP:Q', title = 'Accumulated Precip (mm)'),
    tooltip=[alt.Tooltip('DATE', title="Date"), alt.Tooltip('TIME', title="Time"), alt.Tooltip('APCP', title="Accumulated Precip")]
).properties(width=700, height=130)

In [192]:
precip_total = (precip + precip_accum).resolve_scale(
    y = 'independent'
)

In [193]:
title = alt.TitleParams(
   text='Grouse Mountain (123.08W, 549.38N, Model elevation: 267m) CAN-GDPS',
   subtitle="Air temp (red), dew point temp (blue), temp @1500m (red dashed), RH (green dashed)",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

temp_line = alt.Chart(meteo, title=title).mark_line(color='red', strokeWidth=2).encode(
    alt.X('DATETIME:T',  title=None, axis=alt.Axis(labelAngle=0, format='%e %b')),
    alt.Y('TMP:Q', title='Temperature (C)'),
    tooltip=[alt.Tooltip('DATE', title="Date"), alt.Tooltip('TIME', title="Time"), alt.Tooltip('TMP', title="Air Temp")]
).properties(width=700, height=130)

dew = alt.Chart(meteo).mark_line(color='blue', strokeWidth=2).encode(
    alt.X('DATETIME:T',  title=None, axis=alt.Axis(labelAngle=0, format='%e %b')),
    alt.Y('DPT:Q', title='Temperature (C)')
).properties(width=700, height=130)

zero = alt.Chart(meteo).mark_line(color='gray', strokeWidth=1.5, opacity=0.8, strokeDash=[3,3]).encode(
    alt.X('DATETIME:T',  title=None, axis=alt.Axis(labelAngle=0, format='%e %b')),
    alt.Y('Zero:Q', title='Temperature (C)')
).properties(width=700, height=130)

rh = alt.Chart(meteo).mark_line(color='green', strokeWidth=1.5, opacity=0.7, strokeDash=[3,3]).encode(
    alt.X('DATETIME:T',  title=None, axis=alt.Axis(labelAngle=0, format='%e %b')),
    alt.Y('RH:Q', title='Relative Humidity (%)', scale=alt.Scale(zero=False))
).properties(width=700, height=130)

temp1500 = alt.Chart(meteo).mark_line(color='darkred', strokeWidth=1.5, opacity=0.7, strokeDash=[4,2]).encode(
    alt.X('DATETIME:T',  title=None, axis=alt.Axis(labelAngle=0, format='%e %b')),
    alt.Y('TMP850:Q', title='Temperature (C)'),
    tooltip=[alt.Tooltip('DATE', title="Date"), alt.Tooltip('TIME', title="Time"), alt.Tooltip('TMP850', title="Air Temp")]
).properties(width=700, height=130)

area = alt.Chart(meteo).mark_area(color='#F8D223', opacity=0.5).encode(
    alt.X('DATETIME:T'),
    alt.Y('TMP:Q'),
    alt.Y2('DPT:Q')
).properties(width=700, height=130)

temp1 = temp_line + dew + temp1500 + area +zero

temp2 = alt.layer(temp1, rh).resolve_scale(
    y='independent'
)

In [194]:
pressure = alt.Chart(meteo).mark_line(color='red', strokeWidth=1.5, opacity=0.7).encode(
    alt.X('DATETIME:T',  title=None, axis=alt.Axis(labelAngle=0, format='%e %b')),
    alt.Y('SLP', title='MSLP (mbar)', scale= alt.Scale(zero=False))
).properties(width=700, height=130, title='Cloud cover (blue/gray), sea level pressure (red)')

cloud_line = alt.Chart(meteo).mark_line(color='blue', strokeWidth=1.5, opacity = 0.7).encode(
    alt.X('DATETIME:T',  title=None, axis=alt.Axis(labelAngle=0, format='%e %b')),
    alt.Y('CLOUD', title='Cloud Cover (%)')
).properties(width=700, height=130)

cloud_area = alt.Chart(meteo).mark_area(color='#B4A667', opacity=0.5).encode(
    alt.X('DATETIME:T', axis=alt.Axis(labelAngle=0, format='%e %b')),
    alt.Y('CLOUD:Q')
).properties(width=700, height=130)

cloud = cloud_area + cloud_line
    
cloud_pressure = alt.layer(cloud, pressure).resolve_scale(
    y='independent'
)


In [195]:
wind_750 = alt.Chart(meteo).mark_line(color='red', strokeWidth=1.5).encode(
    alt.X('DATETIME:T',  title=None, axis=alt.Axis(labelAngle=90, format='%e %b')),
    alt.Y('WS925', title='Wind speed (km/h)'),
    tooltip=[alt.Tooltip('DATE', title="Date"), alt.Tooltip('TIME', title="Time"), alt.Tooltip('WS925', title="Wind Speed (km/h)"), alt.Tooltip('WD925', title="Wind Dir (deg)")]
).properties(width=700, height=130, title='Wind speed at 750m (red) and 1500m (green)')

wind_1500 = alt.Chart(meteo).mark_line(color='green', strokeWidth=1.5).encode(
    alt.X('DATETIME:T',  title=None, axis=alt.Axis(labelAngle=90, format='%e %b')),
    alt.Y('WS850', title='Wind speed (km/h)'),
    tooltip=[alt.Tooltip('DATE', title="Date"), alt.Tooltip('TIME', title="Time"), alt.Tooltip('WS850', title="Wind Speed (km/h)"), alt.Tooltip('WD850', title="Wind Dir (deg)")]
).properties(width=700, height=130)

wind = wind_750 + wind_1500 


In [196]:
meteogram = temp2 & cloud_pressure & precip_total & wind 
meteogram = meteogram.configure_axis(labelFontSize=11, titleFontSize=12)
meteogram

alt.VConcatChart(...)

## Temperature Outlooks

The maps below show the probabilities of above, below, or near-normal temperatures for the next 8-14 days, 1-month, and 3-months. This gives us a rough indication of average temperatures over these periods. The higher the likelihood, the more confidence in the forecast. Currently it looks as though mid-October could be relatively cool and wet, but most longer range products continue to call for warmer and drier than normal conditions for the fall and early winter. 

### 8-14 day outlook 
**Issued on October 2.** Temperatures during the middle of October may be cooler than normal for the south coast and Pacific Northwest.

```{figure} img/twoweek_temp.png
---
name: twoweek
---
8-14 day temperature outlook ([ECCC](https://weather.gc.ca/ensemble/naefs/semaine2_combinee_e.html))
```
### One month outlook 
**Issued on September 28.** The monthly temperature outlook shows slightly higher odds of above normal temperatures for the BC south coast in October. 

```{figure} img/1month.gif
---
name: monthly
---
Monthly temperature outlook ([ECCC](https://weather.gc.ca/saisons/image_e.html?img=mfe1t_s))
```
### 1-3 month temperature and precipitation outlooks 

**Issued on September 30.** The 3-month temperature and precipitation favour relatively warm conditions for the BC south coast for the Oct-Nov-Dec period. There is a 60-70% probability of above normal temperatures for this period. There are equal odds of above or below normal precipitation. 

```{figure} img/3month-temp-precip.png
---
name: three-monthly
---
Three month (S-O-N) temperature and precipitation outlooks ([ECCC](https://weather.gc.ca/saisons/prob_e.html))
```
## Summary

Drought conditions should improve in October, even with only average rainfall amounts. Medium-range outlooks favour mild but relatively wet conditions for the middle of October; however, longer range outlooks still show higher odds of above normal temperatures and below normal precipitation. We might see periods of dry and warm weather punctuated by strong fall storms. 